# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109A Introduction to Data Science: 
## Homework 2: kNN and Linear Regression

**Harvard University**<br/>
**Fall 2020**<br/>
**Instructors**: Pavlos Protopapas, Kevin Rader, and Chris Tanner

<hr style="height:2.4pt">

In [ ]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [ ]:
#RUN THIS CELL
import os
import pathlib
working_dir = pathlib.Path().absolute()
os.chdir(working_dir)

<hr style="height:2pt">

### INSTRUCTIONS

- To submit your assignment follow the instructions given in Canvas.

- This homework can be submitted in pairs, and it is encouraged for you to do so. Especially during covid and distancing, this can be a way to work with other students and learn alongside one another. As future data scientists, you will often be expected to work with others, and working in pairs can help practice communicating data science concepts.

- Please restart the kernel and run the entire notebook again before you submit.

- Running cells out of order is a common pitfall in Jupyter Notebooks. To make sure your code works restart the kernel and run the whole notebook again before you submit. Exceptions should be made for code with a long execution time, of course.
- We have tried to include all the libraries you may need to do the assignment in the imports statement at the top of this notebook. We strongly suggest that you use those and not others as we may not be familiar with them. .
- Please use .head() when viewing data. Do not submit a notebook that is **excessively long**. 
- In questions that require code to answer, such as "calculate the $R^2$", do not just output the value from a cell. Write a `print()` function that includes a reference to the calculated value, **not hardcoded**. For example: 
```
print(f'The R^2 is {R:.4f}')
```

<hr style="height:2pt">

<div class="theme"> Overview </div> 

This assignment is the first where you will go through the process of loading a dataset, splitting it in train and test sets, 
pre-processing it, and finally using it to run models and evaluating your results. 

We have two different datasets, one with car data in **Part 1** and another with data from an Indian matrimonial web site in **Part 2**.

For part 1, you will explore two simple methods for prediction,  **k-nearest neighbors regression (kNN)**, a *non-parametric* method, and **linear regression**, a *parametric* method. As you move towards Part 2 of the homework, you will work with multiple linear and polynomial regression.

<div style="color: red; background: black">Please note that Question 4 and Question 7 are required for 209a students but are <strong>optional for 109a students</strong>. We include them here for your education, and we believe that if you have time to spend with them you will learn from them. But <strong>if you are in 109a, then Q4 and Q7 are completely optional</strong>. To help manage stress if you are not in 209a, we recommend skipping past these first and then coming back to them once you have finished the rest of the homework if you have time.</div>

In [ ]:
#!pip install seaborn

In [ ]:
#Importing standard libraries
import numpy as np
import operator
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

# pandas tricks for better display
pd.options.display.max_columns = 50  
pd.options.display.max_rows = 500     
pd.options.display.max_colwidth = 100
pd.options.display.precision = 3

# Part 2 imports 
from scipy.stats import norm
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
# Run this cell for more readable visuals 
large = 22; med = 16; small = 10
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'figure.figsize': (16, 10),
          'axes.labelsize': med,
          'axes.titlesize': med,
          'axes.linewidth': 2,
          'xtick.labelsize': med,
          'ytick.labelsize': med,
          'figure.titlesize': large}
plt.style.use('seaborn-whitegrid')
plt.rcParams.update(params)
#sns.set_style("white")
%matplotlib inline

<div class="theme"> Part A: k-NN and Linear Regression</div> 

### Problem Description: Predicting the Selling Price of Cars on CarDekho.com

According to its website, **CarDekho.com** is India's leading car search venture. 
Its website and app carry rich automotive content such as expert reviews, 
detailed specs and prices, comparisons as well as videos and pictures of all car brands and models available in India. 
Each car has a **Current selling price**, which is the price for buying the car on this site, and a **MRP**, 
which is the retail price of the car. These two prices differ depending on factors such as brand, 
make year, millage, condition, etc.  

#### Data set 

The dataset contains 601 cars and is in file `car_dekho_full.csv`. It contains the following columns:

- **Year** - make year (year the car was made), 
- **Current_Selling_Price** - current price of a car on CarDekho.com (in lakhs),
- **MRP** - maximum retail price of a car (in lakhs). 
- **Kms_Driven** - number of kilometers

Note: 1 *lakh*  is 100,000 Rupees in the Indian numbering system. Also, kilometers are used as a measure of distance instead of miles.

#### Your Task: 
Predict the `Current_Selling_Price` from the other features.

<div class='exercise'><b> Question 1:   Exploratory Data Analysis (EDA) [10 pts]</b>

To reach the goal of predicting the `Current_Selling_Price`, start by inspecting the dataset using Exploratory Data Analysis (EDA).

Load the dataset, inspect it and answer the following questions: 

**1.1** Which variables are quantitative, and which are categorical? 

**1.2** What are the means and standard deviations for `Current_Selling_Price` and `MRP`? 

**1.3** What is the range of Kilometers that the cars have?

**1.4** The goal of this part is to identify the best variable from which to predict our respone variable `Current Selling Price`. Plot a scatter plot between each predictor and reponse variable and examine the relationship between the predictors and `Current_Selling_Price`. Based on the plots, which is the  predictor that visually seems to best predict the `Current_Selling_Price`? 
    
    
Note: In general, it is always good to label your axes, title your graphs, and produce visuals which clearly communicate the data. Visuals should often be accompanied by text identifying the key point of the visual and defending any choices you make as a data scientist regarding the visual to best communicate your data. 
</div>

## Solutions 

<div class='exercise-r'>  
 
**1.1** Which variables are quantitative, and which are categorical?
 
 </div>

In [ ]:
cars = pd.read_csv('data/car_dekho_full.csv')
cars.head()

In [ ]:
print("Year is a categorical data in that it is not continuous, but it is ordered. "
      "Selling price, max retail price, and kilometers driven are "
      "all quantitative variables")

<div class='exercise-r'>  
 
**1.2** What are the means and standard deviations for `Current_Selling_Price` and `MRP`?
 
 </div>

In [ ]:
# name your variables mean_csp, mean_mrp, std_csp, std_mrp
# your code here
df = cars.describe()
mean_csp = df['Current_Selling_Price'].mean()
std_csp = df.loc['std', 'Current_Selling_Price']
mean_mrp = df.loc['mean', 'MRP']
std_mrp = df.loc['std', 'MRP']

print(f"The mean selling price is {mean_csp:.2f} lakhs with a standard deviation of {std_csp:.2f}."
      f"The mean maximum retail price is {mean_mrp:.2f} lakhs with a standard deviation of {std_mrp:.2f}.")

<div class='exercise-r'>  
 
**1.3** What is the range of Kilometers that the cars have?
 
 </div>

In [ ]:
print(f"The car with the lowest number of Km on it has {df.loc['min','Kms_Driven']} km. "
     f"The car with the highest number of Km on it has {df.loc['max', 'Kms_Driven']} km.")

<div class='exercise-r'>  
 
**1.4** The goal of this part is to identify the best variable from which to predict our respone variable `Current Selling Price`. Plot a scatter plot between each predictor and reponse variable and examine the relationship between the predictors and `Current_Selling_Price`. Based on the plots, which is the  predictor that visually seems to best predict the `Current_Selling_Price`?
 
 
 Note: In general, it is always good to label your axes, title your graphs, and produce visuals which clearly communicate the data. Visuals should often be accompanied by text identifying the key point of the visual and defending any choices you make as a data scientist regarding the visual to best communicate your data.
 </div>

In [ ]:
y = cars['Current_Selling_Price']
for x in ['Kms_Driven', 'MRP', 'Year']:
    plt.figure()
    plt.xlabel(x)
    plt.title(f"Relationship between {x} and current selling price")
    plt.ylabel("Current Selling Price")
    plt.scatter(cars[x],y)
    
year_df = cars.groupby(by='Year').mean()
x = "year"
plt.figure()
plt.xlabel(x)
plt.title(f"Relationship between {x} and mean current selling price")
plt.ylabel("Current Selling Price")
plt.scatter(year_df.index, year_df['Current_Selling_Price'])

Based on the graphs above, it seems that the strongest 
predictor of current selling price is maximum retail price, 
although year also seems like a fairly good predictor. Kilometers driven seems to have a slight negative correlation with current selling price, but it is not ver strong. 

Since year is a discrete variable, it is hard to see all of the data points when plotting year vs current selling price. Therefore I chose to also plot the mean selling price for each year. 

<div class='exercise'><b> Question 2:   k-Nearest Neighbors [25 pts]</b>

We begin our modeling with k-Nearest Neighbors (kNN) regression. You may use `sklearn`'s built-in functions.

**2.1** In this part, we will model a kNN regression on the predictor chosen above (1.4) and the response variable `Current_Selling_Price`.

    
INSTRUCTIONS:
- Split the dataset in train and test set with 75% training data and 25% testing data, using the random_state = 109.  
- Fit a kNN regression model to the training set for the following 8 different values of $k$:  $k = 1,2,3,5,7,10,50,100$. 
- Make 8 scatter plots of response vs. predictor for each $k$, arranged in a $4\times2$ grid.  Each figure should have plots of the prediction from the k-NN regression and the actual data points on the same figure, as well as axis labels, title, and legend. Consider using the subplot functionality, unless you first try this and then decide that you have a clearer, cleaner way of communicating these plots. 
- Evaluate the $MSE$ for the fitted models on both the training and test sets **for each $k$**.
- Plot the training and test $MSE$ values as a function of $k$ on the same figure.  Again, the figure must have axis labels and a legend.
- Find the best model based on the test $MSE$ values.
- Evaluate and report the $R^2$ of the best model.

**2.2** Discuss your results by answering the following questions.  You should answer the questions directly in a markdown cell of your notebook.

- How does the value of $k$ affect the fitted model?

- If $n$ is the number of observations in the training set, what can you say about a k-NN regression model that uses $k = n$?  

- Do the training and test $MSE$ plots exhibit different trends?  Explain how the value of $k$ influences the training and test $MSE$ values.

- Run the same code by changing the random seed during the train-test split. Do you always get the same answer? If not, why?
    
    </div>

### Solutions

<div class='exercise-r'>  
 
**2.1** In this part, we will model a kNN regression on the predictor chosen above (1.4) and the response variable `Current_Selling_Price`.
 
 
 INSTRUCTIONS:
 - Split the dataset in train and test set with 75% training data and 25% testing data, using the random_state = 109.
 - Fit a kNN regression model to the training set for the following 8 different values of $k$:  $k = 1,2,3,5,7,10,50,100$.
 - Make 8 scatter plots of response vs. predictor for each $k$, arranged in a $4\times2$ grid.  Each figure should have plots of the prediction from the k-NN regression and the actual data points on the same figure, as well as axis labels, title, and legend. Consider using the subplot functionality, unless you first try this and then decide that you have a clearer, cleaner way of communicating these plots.
 - Evaluate the $MSE$ for the fitted models on both the training and test sets **for each $k$**.
 - Plot the training and test $MSE$ values as a function of $k$ on the same figure.  Again, the figure must have axis labels and a legend.
 - Find the best model based on the test $MSE$ values.
 - Evaluate and report the $R^2$ of the best model.
 
 </div>

In [ ]:
#predictor - MRP
#response - Current Selling Price
x = cars[['MRP']]
y = cars['Current_Selling_Price']

In [ ]:
##Splitting the data into train and test sets with 75% training data and 25% testing data. Set random_state=109
# your code here
x_train, x_test, y_train, y_test = train_test_split(x, y, 
        train_size = 0.75, random_state = 109)

## <font color=#FF00FF>NEED TO FIX THE PLOTTING HERE</font>
Check that 

In [ ]:
## Fit a kNN regression model to the training set for the following 8 different values of  𝑘 :  𝑘=1,2,3,5,7,10,50,100 .
## and make 8 scatter plots of response vs. predictor for each  𝑘 , arranged in a  4×2  grid. 
## Each figure should have plots of the prediction from the k-NN regression and the actual data points on the same figure, as well as axis labels, title, and legend
# your code here 

k_list = [1,2,3,5,7,10,50,100]
MSE_train = {}
MSE_test = {}

plt.scatter(cars['MRP'], cars['Current_Selling_Price'])

for k in k_list:
    #fit the model
    model = KNeighborsRegressor(n_neighbors = k)
    model.fit(x_train, y_train)
    
    #sort the values so I can plot them nicely
    #not sorting them also lead to very bad MSE behavior
    #because I was comparing a sorted y_pred with an unsorted
    #y_test!!
    
    x_test = x_test.sort_values(by='MRP')
    y_test = y_test.loc[x_test.index]
    
    #test on testing data
    y_pred = model.predict(x_test)
    MSE_test[k] = mean_squared_error(y_test, y_pred)
    
    #test on training data
    y_train_pred = model.predict(x_train)
    MSE_train[k] = mean_squared_error(y_train, y_train_pred)
    
    #plot the fits
    plt.plot(x_test,y_pred, '.-', label = f'k = {k}')
    plt.legend()

In [ ]:
# Now make MSE plots
plt.plot(k_list, [*MSE_test.values()],'k.-', label = 'Testing Data')
plt.plot(k_list, [*MSE_train.values()],'b.-', label = 'Training Data')

plt.xlabel('k',fontsize=20)
plt.ylabel('MSE',fontsize = 20)
plt.title('$MSE$ values vs k values - KNN regression',fontsize=20)
plt.legend()

In [ ]:
# Find the best model
min_index = np.argmin([*MSE_test.values()])
best_k = k_list[min_index]
best_MSE = min(MSE_test.values())
print(f"The model with the lowest MSE value of {best_MSE:.2f} has a k of {best_k}.")

In [ ]:
##Compute the R-squared for the best model
best_model = KNeighborsRegressor(n_neighbors=best_k)
best_model.fit(x_train, y_train)
best_y_pred = best_model.predict(x_test)
best_r = r2_score(y_test, best_y_pred)
print(f"The R^2 score of the best model is {best_r:.3f}")

<div class='exercise-r'>  
 
**2.2** Discuss your results by answering the following questions.  You should answer the questions directly in a markdown cell of your notebook.
 
 - How does the value of $k$ affect the fitted model?
 
 - If $n$ is the number of observations in the training set, what can you say about a k-NN regression model that uses $k = n$?
 
 - Do the training and test $MSE$ plots exhibit different trends?  Explain how the value of $k$ influences the training and test $MSE$ values.
 
 - Run the same code by changing the random seed during the train-test split. Do you always get the same answer? If not, why?
 
 </div>

- The value of k affects the fitted model by choosing how many nearest neighbors it will look at. If the k is low, the model may overfit to the trianing data. If the k is high, the model can average out real trends.

- If the $k = n$, then the model is just taking the average of all the data points, the value of the predictor will not affect the prediction at all.

- The MSE vs k plots look different for training and testing data. Lower k values look much better for training data than they do for testing data. This makes sense because the low k is essentially overfitting to the training data, so of course it will perform well with that data. 

- No. When I change the random seed to 209 and then to 100, I get 'best model k values' of 1 and 5 respectively. Perhaps the fitting function is not a convex function so the graident descent is getting caught it local minima depending on where the random state starts the algorithm in.

## <font color = #FF00FF> Why is this? </font>

<div class='exercise'><b> Question 3:  Simple Linear Regression [25 pts]</b>

**3.1** We will now fit our data using a linear regression model. Choose the same **predictor** and **response** variables used to model kNN regression.

- You will use the same 75% training and 25% testing split of the data, using the same random_state = 109. 
- Run a Linear Regression model.
- Report the slope/coefficient and intercept values for the fitted linear model.
- Report the $MSE$ for the training and test sets and the $R^2$ from the test set.
- Plot the **residuals** $e = y - \hat{y}$ of the model on the training set as a function of the response variable. Draw a horizontal line denoting the zero residual value on the Y-axis.

**Note:** Use the `sklearn` module for linear regression. This module has built-in functions to summarize the results of regression and produce residual plots. Create a `Linear Regression` model, use the `fit` method in the instance for fitting a linear regression model, and use the `predict` method to make predictions. As previously, you may use the `mean_squared_error` function to compute $MSE$.
    
**3.2** Discuss your results by answering the following questions.  

- How does the test $MSE$ score compare with the best test $MSE$ value obtained with k-NN regression? 

- What does the sign of the slope of the fitted linear model convey about the relationship between the predictor and the response?

- Discuss the shape of the residual plot and what it shows for the quality of the model. Be sure to discuss whether or not the assumption of linearity is valid for this data.
    </div>


### Solutions

<div class='exercise-r'>  
 
**3.1** We will now fit our data using a linear regression model. Choose the same **predictor** and **response** variables used to model kNN regression.
 
 - You will use the same 75% training and 25% testing split of the data, using the same random_state = 109.
 - Run a Linear Regression model.
 - Report the slope/coefficient and intercept values for the fitted linear model.
 - Report the $MSE$ for the training and test sets and the $R^2$ from the test set.
 - Plot the **residuals** $e = y - \hat{y}$ of the model on the training set as a function of the response variable. Draw a horizontal line denoting the zero residual value on the Y-axis.
 
 **Note:** Use the `sklearn` module for linear regression. This module has built-in functions to summarize the results of regression and produce residual plots. Create a `Linear Regression` model, use the `fit` method in the instance for fitting a linear regression model, and use the `predict` method to make predictions. As previously, you may use the `mean_squared_error` function to compute $MSE$.
 
 </div>

In [ ]:
# Prepare your data
x_train, x_test, y_train, y_test = train_test_split(x, y, 
        train_size = 0.75, random_state = 109)

x_test = x_test.sort_values(by='MRP')
y_test = y_test.loc[x_test.index]

In [ ]:
## Fit a linear model to the train data
model = LinearRegression()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
y_train = model.predict(x_train)

#plot the data just to make sure everything looks good
plt.plot(x_test,y_pred, 'k')
plt.scatter(x_test, y_test)
plt.title("Liner regression model")
plt.xlabel("MRP (lakhs)")
plt.ylabel("Current selling price (lakhs)")

In [ ]:
## Report the slope/coefficient and intercept values for the fitted linear model. 
print(f"The slope of the model is {model.coef_[0]:.2f} and the "
      f"intercept is {model.intercept_:.2f}.")

In [ ]:
## Report the $MSE$ and $R^2$ from the training and test sets.
MSE = mean_squared_error(y_test, y_pred)
R2 = r2_score(y_test, y_pred)
print(f"For the test data the MSE value is {MSE:.2f}, and the R2 is {R2:.2f}")

In [ ]:
## Plot the **residuals** 
residuals = y_test - y_pred
plt.hlines(y=0, xmin = min(y_test)-1, xmax = max(y_test)+1)
plt.scatter(y_pred, residuals)
plt.title("Residual vs response variable")
plt.xlabel("Predicted Current Sales Price (lakhs)")
plt.ylabel("Residual (lakhs)")

<div class='exercise-r'>  
 
**3.2** Discuss your results by answering the following questions.
 
 - How does the test $MSE$ score compare with the best test $MSE$ value obtained with k-NN regression?
 
 - What does the sign of the slope of the fitted linear model convey about the relationship between the predictor and the response?
 
 - Discuss the shape of the residual plot and what it shows for the quality of the model. Be sure to discuss whether or not the assumption of linearity is valid for this data.
 </div>

- The best test MSE I got with k-NN regression was 2.85 (with a random seed of 100). The linear MSE is slightly higher, but not much.
- The slope signifies how much the selling price will increase in lakhs for every one increase in lahks of the maximum retail price.
- There does not seem to be any non-linear predicted y dependence in the residuals. It is clear that the model is worse for higher predicted sales price values, but the linear model seems valid.

<div class='exercise'><b> Question 4 (for 209a students, optional for others):  Linear Regression with Feature engineering [10 pts]</b>

**4.1** Creating a new variable from existing data: percentage depreciation

Feature engineering involves transforming data into features that better represent the underlying problem to the predictive models. This results in improved model accuracy on unseen data. 

Our previous regression model relates the Current selling price to the MRP of the car with the equation:

$$CSP = \beta_0 + \beta_1*MRP$$

However, this linear equation does not incorparate other interesting variables such as the ```year of manufacture```, or the ```kms driven```, which may be important factors that affect the overall price of the car. 

Instead of using multi-linear analysis, we can perform some intelligent feature engineering to identify other simple linear relationships within our data.

From practical experience, we know that the percentage drop of a car's price is proportional to the age of the car ([more on car depreciation here](https://www.finder.com/what-is-car-depreciation)). 

Hence, it makes sense to investigate this variable seperately and try to identify possible relationships with other variables.  

Define the percentage depreciation of the Current selling price to the MRP as follows:

$$\textrm{Percentage of the Selling Price}=perc =\frac{MRP - Selling Price}{MRP}$$
    
Create a new column in your dataframe for `perc`
    
    
**4.2** Exploratory Data Analysis

For this section, we will consider `perc` to be our intermediate response variable. To understand the relationship between `perc` and our predictor variables we will perform EDA.

Answer the following questions by plotting graphs.

a) It is seen previosuly that there is a relationship between `Year` and `Current Selling Price`. Is the relationship between `Years` and `perc` the same. If not, how has it changed and why do you think so?

b) Is the trend between the `MRP` and `perc` the same as that between `MRP` and `Current Selling Price`?

c) Does there seem to be a relationship between `Kms_Driven` and `perc` ? 

d) Which is the best predictor to predict `perc`, if there is one? Is it the same as that of `Current Selling price` or has it changed?

**4.3** Perform additional EDA 

Use other plots and statistics to find the best predictor and/or understand the relationship between the other variables and `perc`. One example is given below. It is a plot of `perc` vs `Year` that is color coded based on the `Kms_Driven`.

**4.4** Fitting a Linear Regression model

Based on the previous EDA choose appropriate **feature** variable(s) and **response** variable (`perc`).

- Again, use the same split train-test sets with training data of 75% and testing data of 25%
- Fit a Linear Regression model for each of the predictors.
- Predict the model for the train and test data
- Plot a graph with the test data with predictor variable on the *x* axis and `perc` on the *y* axis. Also plot the fit curve. Ensure you use the correct labels and show the legend.
- Report the $MSE$ score from the training and test sets.
- Find the best model i.e. the best predictor based on the $MSE$ of each model.

**4.5** Predicting The Current Selling Price using ```perc``` 

After performing the above analysis, answer briefly as to why are we getting such a dramatic increase in the R2 score.
    
 </div>

## Solutions

<div class='exercise-r'>  
 
**4.1** Creating a new variable from existing data: percentage depreciation
 
 Feature engineering involves transforming data into features that better represent the underlying problem to the predictive models. This results in improved model accuracy on unseen data.
 
 Our previous regression model relates the Current selling price to the MRP of the car with the equation:
 
 $$CSP = \beta_0 + \beta_1*MRP$$
 
 However, this linear equation does not incorparate other interesting variables such as the ```year of manufacture```, or the ```kms driven```, which may be important factors that affect the overall price of the car.
 
 Instead of using multi-linear analysis, we can perform some intelligent feature engineering to identify other simple linear relationships within our data.
 
 From practical experience, we know that the percentage drop of a car's price is proportional to the age of the car ([more on car depreciation here](https://www.finder.com/what-is-car-depreciation)).
 
 Hence, it makes sense to investigate this variable seperately and try to identify possible relationships with other variables.
 
 Define the percentage depreciation of the Current selling price to the MRP as follows:
 
 $$\textrm{Percentage of the Selling Price}=perc =\frac{MRP - Selling Price}{MRP}$$
 
 Create a new column in your dataframe for `perc`
 
 
 </div>

In [ ]:
cars['perc'] = (cars['MRP'] - cars['Current_Selling_Price'])/cars['MRP']

<div class='exercise-r'>  
 
**4.2** Exploratory Data Analysis
 
 For this section, we will consider `perc` to be our intermediate response variable. To understand the relationship between `perc` and our predictor variables we will perform EDA.
 
 Answer the following questions by plotting graphs.
 
 a) It is seen previosuly that there is a relationship between `Year` and `Current Selling Price`. Is the relationship between `Years` and `perc` the same. If not, how has it changed and why do you think so?
 
 b) Is the trend between the `MRP` and `perc` the same as that between `MRP` and `Current Selling Price`?
 
 c) Does there seem to be a relationship between `Kms_Driven` and `perc` ?
 
 d) Which is the best predictor to predict `perc`, if there is one? Is it the same as that of `Current Selling price` or has it changed?
 
 </div>

a)

In [ ]:
def EDA(x, y, x_label = None, y_label = None):
    plt.figure()
    plt.scatter(cars[x], cars[y])
    if x_label:
        plt.xlabel(x_label)
    else:
        plt.xlabel(x)
    if y_label:
        plt.ylabel(y_label)
    else:
        plt.ylabel(y)
    


EDA('Year', 'Current_Selling_Price')
EDA('Year', 'perc', y_label = 'Percent Derpecation')

Current selling price generally increases with the year, wheras precentage deprecation decreases with year. This makes sense, because if the cars started out the same price, if the price deprecates the selling price will be lower.

b)

In [ ]:
EDA('MRP', 'Current_Selling_Price')
EDA('MRP', 'perc', y_label = 'Percentage Deprecation')

The relationship between MRP and current selling price is NOT the same as the relationship between MRP and precentage deprecation. There does not seem to be a strong correlation between MRP and precentage deprecation, wheras there is a strong relationship between MRP and current selling price.

c)

In [ ]:
EDA('Kms_Driven', 'perc', y_label = "Percentage Deprecation")

There is a correlation between kilometers driven and percentage deprecation. Generally the more kilometers a car has been driven, the higher the deprecation. 

d)

It seems that both year and kilometers driven are good indicators of the percentage deprecation. 

<div class='exercise-r'>  
 
**4.3** Perform additional EDA
 
 Use other plots and statistics to find the best predictor and/or understand the relationship between the other variables and `perc`. One example is given below. It is a plot of `perc` vs `Year` that is color coded based on the `Kms_Driven`.
 
 </div>

In [ ]:
newdf = cars.copy()

newdf['bins'] = np.int64(newdf.Kms_Driven/30000)

bins = newdf.bins.unique()

bins.sort()

fg = sns.FacetGrid(data= newdf, hue = 'bins',hue_order = bins,
                  height = 6, aspect = 1.61)
fg.map(plt.scatter, 'Year','perc')
plt.legend(['<30k','30-60k','60-90k','90-120k','120-150k','150-180k','180k+'],loc='lower left');

In [ ]:
EDA('Current_Selling_Price', 'perc') # MRP


There is a slight negative correlation between current selling price and precentage deprecation. Not as strong as Kms driven or year though. This implies that if the current selling price is very high, it is more likely that the precentage of deprecation is low, which makes sense because if an car that is being sold for a lot of money has already deprecated a lot, it must have started out even more expensive. 

In [ ]:
years = np.sort(cars['Year'].unique())
fg = sns.FacetGrid(data= cars, hue = 'Year', height = 6, aspect = 1.61, hue_order = years)
fg.map(plt.scatter, 'Kms_Driven','perc')
plt.legend(years, loc = 'right');

The above graph shows the same data as the provided example but with the hue and x axes switched. It seems that since kms driven and year correlated, I am not sure how useful using both will be.

<div class='exercise-r'>  
 
**4.4** Fitting a Linear Regression model
 
 Based on the previous EDA choose appropriate **feature** variable(s) and **response** variable (`perc`).
 
 - Again, use the same split train-test sets with training data of 75% and testing data of 25%
 - Fit a Linear Regression model for each of the predictors.
 - Predict the model for the train and test data
 - Plot a graph with the test data with predictor variable on the *x* axis and `perc` on the *y* axis. Also plot the fit curve. Ensure you use the correct labels and show the legend.
 - Report the $MSE$ score from the training and test sets.
 - Find the best model i.e. the best predictor based on the $MSE$ of each model.
 
 </div>

In [ ]:
# split up the data
def lin_model(x, y):
    x_train, x_test, y_train, y_test = train_test_split(cars[[x]], cars[y], train_size = 0.75,
                                                   random_state = 100)
    
    model = LinearRegression()
    model.fit(x_train, y_train)
    y_pred_test = model.predict(x_test)
    y_pred_train = model.predict(x_train)
    
    plt.figure()
    plt.plot(x_train, y_pred_train, label = "Train Pred")
    plt.scatter(x_train, y_train, label = "Train Data")
    plt.plot(x_test, y_pred_test, 'k', label = "Test Pred")
    plt.scatter(x_test, y_test, label = "Test Data")
    plt.xlabel(x)
    plt.ylabel(y)
    plt.legend()
    plt.show()
    
    MSE_train = mean_squared_error(y_test, y_pred_test)
    MSE_test = mean_squared_error(y_train, y_pred_train)
        
    print(f"Using {x} as a predictor, the MSE on the test data "
          f"is {MSE_test:.2f}, and the MSE on the train data "
          f"is {MSE_train:.2}.")
    return model

In [ ]:
kms_perc_model = lin_model('Kms_Driven', 'perc')
year_perc_model =lin_model('Year', 'perc')

<div class='exercise-r'>  
 
**4.5** Predicting The Current Selling Price using ```perc```
 
 After performing the above analysis, answer briefly as to why are we getting such a dramatic increase in the R2 score.
 
 </div>

In [ ]:
B0 = year_perc_model.intercept_
B1 = year_perc_model.coef_

#combine the two equations we have to calucuate percent deprecation 
#to get a new way to predict CSP 
csp_pred = -(B0+B1*cars['Year'])*cars['MRP']+cars['MRP']

#calculate the R^2 term
R2 = r2_score(cars['Current_Selling_Price'], csp_pred)

print(f"My new model gives an R^2 of {R2:.2f}, while using only MRP, as in problem 3.1, gives an R^2 of only 0.81")

The new model is a linear regression that also includes an interaction term between year and MRP. 
We likely could have done this without using the perc equations and just directly fit an equation such as 

CSP = B0 + B1\*MRP + B2\*Year + B3\*MRP\*Year

but using the perc variable gives a more real-world tie-in between the two variables.

<div class="theme"> Part Β :  Multi-Linear Regression</div> 

### Problem Description: 

Analysis of publically available profiles on simplymarry.com to learn more about the biases, income disparity & other interesting trends in India. 

#### Dataset

The dataset was aggregated from the SimplyMarry.com site and given in `Income_prediction.csv'.

All the attributes refer to traits and preferences of the person looking for a spouse. 

- **age** - Age of person looking for a spouse
- **gender** - Female:0, Male:1 
- **height** - Height in inches
- **bmi** - BMI calculated based on height and weight
- **eating** - {'Doesn't Matter':0, 'Jain': 1, 'Vegetarian': 2, 'Vegetarian With Eggs': 3, 'Non Vegetarian': 4}
- **family_type** - ('Doesn't Matter': 0, 'Others':3, 'Nuclear': 1, 'Joint family both parents': 2, 'Joint family only mother':2, 'Joint family father mother and brothers sisters':2, 'Joint family single parent brothers and or sisters':2, 'Joint family only father': 2)
- **status** - If social status matters to the person looking for a spouse: {'Doesn't Matter': 0, 'Middle Class': 1, 'Upper Middle Class': 2, 'High Class': 3, 'Rich / Affluent': 4}
- **manglik** - {'No': 0, 'Yes': 1, 'Do Not Know': 2} ([More on this feature](https://en.wikipedia.org/wiki/Mangala_Dosha))
- **drinking** - {'Doesn't Matter':0, 'No': 1, 'Occasionally': 2, 'Yes': 3}
- **complexion** - {'Very Fair ': 1, 'Fair ': 2, 'Wheatish ':3, 'Wheatish Medium ': 4, 'Dark':5}
- **body** - {'Slim': 1, 'Average': 2, 'Heavy': 3, 'Athletic': 4}
- **education** - {'High School':0, 'Some college':1,'Undergrad':2, 'Grad':3, 'Doctorate':4}
- **city** - ('International': 1, 'Mumbai': 2, 'Delhi':3, 'Kolkata':4,'Bengaluru':5, 'Chennai':6, 'Hyderabad':7, 'Pune':8, 'Ahmedabad':9,'Surat':10, 'Vishakapatnam':11, 'Others':12)
- **income** - {Annual income in dollars}

*source: Harvard IACS*

#### Sensitive attributes in the data

It is thought that users are mostly sincere when stating their preferences about their desired partner, and are less likely to hide any deeply held cultural or sociological biases or preferences in order to be perceived as being politically or socially "correct". This might take care of the problem with surveys where responses touching on social norms are notorious for self-report bias, referred to as "social desirability bias." However, the possibility of bias persists; it might be possible to imagine somebody selecting that drinking "doesn't matter" but they might still have some type of preference, unconscious or conscious. 

This is a dataset designed to help us think about issues of bias and social issues in datasets. We hope that you will be able to derive insights into the above mentioned sociological biases. The data could potentially provide answers to interesting questions with associated policy ramifications, such as a possible relationship between bias and factors like education, local environment, or age.


<div class='exercise'><b> Question 5:   Using Data science to learn more about Indian society [25 pts]</b>

First we are going to use simple analytics to learn more about Indian society with the help of this dataset.

The idea is to use basic modeling based on averages & sample distributions to uncover suspected biases, such as gender, skin tone & manglik status.

Answer the below questions using plots & simple statistics.

**5.1** Is there a disparity in income of participants by gender? Consider using a log scale or another technique to communicate clearly.

**5.2** Is there a relationship between income and the "eating" variable? Is there a relationship between income and skin complexion? It is possible to consider skin complexion as an ordinal variable; consider whether retaining this ordering as in the dataset might be preferable to considering skin complexion as a categorical variable lacking order. 

**5.3** Is there a discernable trend in the incomes of participants from different regions/cities?

**5.4** Is there a clear trend between BMI and the income?

**5.5** Does the level of education show a clear trend with income? Is the trend similar across both levels of the "gender" variable available in this dataset?

**5.6** Do any of the numeric attributes show a clear non-linear dependence with the amount of income?

**5.7** Is the income lower or high for those living in 'nuclear' families?

**5.8** What is the average effect of the 'Manglik' variable on income?
    
</div>

### Solutions

<div class='exercise-r'>  
 
**5.1** Is there a disparity in income of participants by gender? Consider using a log scale or another technique to communicate clearly.
 
 </div>

In [ ]:
df_2 = pd.read_csv('data/Income_prediction.csv')
df_2.columns

def rename_var(df, col, var_dict):
    new = 'n_' + col
    df[new] = df[col]
    for key,val in var_dict.items():
        df.loc[df[col]==val,new] = key
    return df

df_2 = rename_var(df_2, 'gender', {'female':0, 'male':1})
df_2 = rename_var(df_2, 'eating', {'Doesnt Matter':0, 'Jain': 1, 'Vegetarian': 2, 'Vegetarian With Eggs': 3, 'Non Vegetarian': 4})
df_2 = rename_var(df_2, 'status', {'Doesnt Matter': 0, 'Middle Class': 1, 'Upper Middle Class': 2, 'High Class': 3, 'Rich / Affluent': 4})
df_2 = rename_var(df_2, 'manglik', {'No': 0, 'Yes': 1, 'Do Not Know': 2})
df_2 = rename_var(df_2, 'drinking', {'Doesnt Matter':0, 'No': 1, 'Occasionally': 2, 'Yes': 3})
df_2 = rename_var(df_2, 'complexion',{'Very Fair ': 1, 'Fair ': 2, 'Wheatish ':3, 'Wheatish Medium ': 4, 'Dark':5})
df_2 = rename_var(df_2, 'education',{'High School':0, 'Some college':1,'Undergrad':2, 'Grad':3, 'Doctorate':4})
df_2 = rename_var(df_2, 'city',{'International': 1, 'Mumbai': 2, 'Delhi':3, 'Kolkata':4,'Bengaluru':5, 'Chennai':6, 'Hyderabad':7, 'Pune':8, 'Ahmedabad':9,'Surat':10, 'Vishakapatnam':11, 'Others':12})
df_2 = rename_var(df_2, 'family_type',{'Doesnt Matter': 0, 'Others':3, 'Nuclear': 1, 'Joint family': 2})

In [ ]:
f, ax = plt.subplots()
ax.set(xscale = 'log')
sns.kdeplot(data=df_2[df_2.n_gender == 'male'], x = "income", ax = ax,
           label = 'female')
sns.kdeplot(data=df_2[df_2.n_gender == 'female'], x = "income", ax = ax, 
            color = 'orange', label = 'male')
f.legend(loc = 'right')
f.show()

In [ ]:
f, ax = plt.subplots()
ax.set(yscale = 'log')
sns.boxplot(data=df_2, x = "n_gender", y = "income", ax = ax)
f.show()

There seems to be somewhat of an income disparity. Generally each of the quartiles is lower for women than for men, and comparing the density function of each gender, there are more women below 10^4 than men, while there are more men making over 10^4 than women.

<div class='exercise-r'>  
 
**5.2** Is there a relationship between income and the "eating" variable? Is there a relationship between income and skin complexion? It is possible to consider skin complexion as an ordinal variable; consider whether retaining this ordering as in the dataset might be preferable to considering skin complexion as a categorical variable lacking order.
 
 </div>

In [ ]:
# your code here 
f, ax = plt.subplots()
ax.set(yscale = 'log')
g = sns.boxplot(data=df_2, x = "n_eating", y = "income", ax = ax)
g.set_xticklabels(g.get_xticklabels(), rotation = 75)
f.legend(loc = 'upper right')
f.show()

There is not a big difference in incomes between vegetarain, vegetarian with eggs, and non-vegetarians. "Doesn't matter" diets have a higher minimum income but the other quartiles seem the same. There are fewer outliers at a highe income though. 

The biggest difference is for Jain diets. The income spead is smaller- the minimim is higher than for most other diets but the median and maximum are both lower and there are fewer outliers. 

In [ ]:
f, ax = plt.subplots()
ax.set(yscale = 'log')
g = sns.boxplot(data=df_2, x = "n_complexion", y = "income", ax = ax)
g.set_xticklabels(g.get_xticklabels(), rotation = 75)
f.legend(loc = 'upper right')
f.show()

Although the range and even the interquartile range is about the same for all complexions, the median gets higher and higher with lighter complexions. People with 'very fair' complexions have higher incomes in every quartile. 

<div class='exercise-r'>  
 
**5.3** Is there a discernable trend in the incomes of participants from different regions/cities?
 
 </div>

In [ ]:
f, ax = plt.subplots()
ax.set()
g = sns.barplot(data=df_2, x = "n_city", y = "income", ax = ax,
               estimator = np.mean)
g.set_xticklabels(g.get_xticklabels(), rotation = 75)
f.legend(loc = 'upper right')
f.show()

In [ ]:
f, ax = plt.subplots()
ax.set(yscale='log')
g = sns.boxplot(data=df_2, x = "n_city", y = "income", ax = ax)
g.set_xticklabels(g.get_xticklabels(), rotation = 75)
f.legend(loc = 'upper right')
f.show()

Cities have a large difference in average income. There are many outliers for each city category so it is easier to interpret only looking at the mean or median and not the entire boxplot. 

'International', Bengaluru, Mumbai, Kolkata, and Deli have the highest mean incomes, wheras 'others', Surat, Vishakapatnam, and Ahmedabad have low mean incomes. Some areas have a much tighter distribution than others. 

<div class='exercise-r'>  
 
**5.4** Is there a clear trend between BMI and the income?
 
 </div>

In [ ]:
f, ax = plt.subplots()
ax.set()
#sort bmi into bins
df_2['bmi_bin'] = np.int64(df_2['bmi']/3)
#drop the bins with too few people in them
df_mod = df_2.loc[(df_2.bmi_bin>3)&(df_2.bmi_bin<11)]
g = sns.barplot(data=df_mod, x = "bmi_bin", y = "income", ax = ax,
               estimator = np.mean)
ax.set_xlabel("BMI/3")
f.show()
print("the number of people in each bin:")
df_2.groupby(by='bmi_bin').count()['age']

Bins 3,11,12,and 13 have too few people in them to consider. It seems that there is not a very strong relationship between BMI and income, although it does seem like having a BMI of ~12 or between ~21-24 is correlated with a higher mean income.

<div class='exercise-r'>  
 
**5.5** Does the level of education show a clear trend with income? Is the trend similar across both levels of the "gender" variable available in this dataset?
 
 </div>

In [ ]:
f, ax = plt.subplots()
ax.set(yscale = 'log')
g=sns.boxplot(data=df_2, x = "n_education", y = "income", ax = ax,
           hue = 'n_gender')
g.set_xticklabels(g.get_xticklabels(), rotation = 75)
f.show()

<div class='exercise-r'>  
 
**5.6** Do any of the numeric attributes show a clear non-linear dependence with the amount of income?
 
 </div>

Having a higher level of education correlate with higher income for both women and men. For both genders there is not a big difference in income between people with some college and an undergraduate degree. In all cases, men earn more on average than women with the same education level.

In [ ]:
# your code here


In [ ]:
# your code here


<div class='exercise-r'>  
 
**5.7** Is the income lower or high for those living in 'nuclear' families?
 
 </div>

In [ ]:
f, ax = plt.subplots()
ax.set(yscale = 'log')
g = sns.boxplot(data=df_2, x = "n_family_type", y = "income", ax = ax)
g.set_xticklabels(g.get_xticklabels(), rotation = 75)
f.show()

There is not a big difference in income between those living in nuclear families and those living in joint or 'other' families.

<div class='exercise-r'>  
 
**5.8** What is the average effect of the 'Manglik' variable on income?
 
 </div>

In [ ]:
f, ax = plt.subplots()
ax.set(yscale = 'log')
g=sns.boxplot(data=df_2, x = "n_manglik", y = "income", ax = ax)
f.show()

There is no effect of the 'Manglik' variable on income.

<div class='exercise'><b> Question 6:  Calculate the Gini Index [15 pts]</b>


Gini coefficients are often used to quantify income inequality, read more [here](http://www.statsdirect.com/help/default.htm#nonparametric_methods/gini.htm).

The Gini coefficient is defined by the formula:

$G = \dfrac{ \sum_{i=1}^{n} (2i - n - 1) x_i}{n  \sum_{i=1}^{n} x_i}$

where $x$ is an observed value, $n$ is the number of values observed and $i$ is the rank of values in **ascending** order.

A Gini Index of 0 implies perfect income equality, whereas a gini index close to 1 implies a concentration of wealth among the richest few.

**6.1** Based on the above formula, calculate the Gini coffient for the income of the participants of this dataset

**6.2** Compare your gini index with other countries.

According to the [world bank estimate](https://www.indexmundi.com/facts/indicators/SI.POV.GINI/rankings) the gini index of South Africa is 0.6 while that of Ukrain is 0.25. 

Based on your calculated Gini index value for this dataset, what is your conclusion on the relationship of the income disparity in the three countries?

Do the data source, self-report nature of the data, or sampling procedure affect your conclusions? If so, how?
    
</div>

### Solutions

<div class='exercise-r'>  
 
**6.1** Based on the above formula, calculate the Gini coffient for the income of the participants of this dataset
 
 </div>

In [ ]:
#number of observations
n = df_2['income'].count()
sum_income = df_2['income'].sum()

#sort by income
df_sorted = df_2.sort_values(by='income').reset_index()

#assign a rank to each income
income_vals = df_sorted['income'].unique()
df_sorted['i'] = df_sorted.index
df_sorted['gini_num'] = (2*df_sorted['i'] - n - 1)*df_sorted['income']
gini_index = df_sorted['gini_num'].sum()/(n*sum_income)


print(f'The Gini index of this dataset is {gini_index:.2f}.')

<div class='exercise-r'>  
 
**6.2** Compare your gini index with other countries.
 
 According to the [world bank estimate](https://www.indexmundi.com/facts/indicators/SI.POV.GINI/rankings) the gini index of South Africa is 0.6 while that of Ukrain is 0.25.
 
 Based on your calculated Gini index value for this dataset, what is your conclusion on the relationship of the income disparity in the three countries?
 
 Do the data source, self-report nature of the data, or sampling procedure affect your conclusions? If so, how?
 
 </div>

The world bank reports that the Gini index of India is 0.37, whereas the index from this dataset is 0.5. Since we are not looking at the entire population of India, it is not surprising that there is some difference between them.

It seems that the people in this dataset have a greater income disparity than Ukrain does but a lower income disparity than Sourth Africa.


<div class='exercise'><b> Question 7 (for 209a students, optional for others):  Multi-Linear Regression [10 pts]</b>

Now we increase the scope of our analysis to solve another problem that is related to income of the participants.
</div>

![](https://github.com/hargun3045/blog-dump/blob/master/modi.png?raw=true)

Owing to a large number of people underreporting their income to evade taxes, the Income Tax Department of India wants you, an esteemed data scientist, to build a machine learning model that can predict the income of a given tax-payer based on commonly available information.

This will help the department red flag suspected individuals who may show discernable trends of earing high values of income but are excessively under-reporting on their annual income.

The goal is to build the best model with the given dataset, using both categorical and continuous predictors that are available.

As with all other homework problems, this is a learning exercise; in the real world, it is your decision to choose the types of data science projects you will work on as well as which clients you will work with. 

Fit a multiple linear regression model to the training set.
Use the `sklearn` library.

#### Deliverables
Your code should be contained in a Jupyter notebook cell.  An appropriate level of comments is necessary.  Your code should run and output the required outputs described below.

#### Required Outputs
- Fit a multiple linear regression model on the training set
- Predict on train and test sets
- Calculate the MSE for the train & test set
- Report the $R^2$ score on the test set.
- Make a plot of Residuals vs Log of predicted values $\hat{y}$, with residuals on the $Y$-axis and predicted values on the $X$-axis. Use the formula ${\epsilon} = y - \hat{y}$ to compute the residual values. Include a horizontal line denoting the zero residual value on the $Y$-axis.
- Plot a histogram of the magnitudes of the residuals.

#### Optional Outputs
You are encouraged to experiment with ways to improve your model *after first reporting results with only the required outputs*. Some ideas are given below:
- Polynomial terms for continous variables
- Interaction terms between variables
- Feature selection among given predictors


In [ ]:
# your code here
We should do this together!